In [63]:
# Need to install NGLView to visualize problem molecules--the rest runs with the yammbs environment
from openff.qcsubmit.results import TorsionDriveResultCollection, OptimizationResultCollection

# suppress stereochemistry warnings
import logging
logging.getLogger("openff").setLevel(logging.ERROR)

from openff.qcsubmit.results.filters import SMARTSFilter
import numpy as np
from openff.toolkit import Molecule

In [2]:
def get_mol_from_recordid(records_and_mols,id_list):
    return [rm for rm in records_and_mols if int(rm[0].id) in id_list]

In [3]:
industry_benchmark = OptimizationResultCollection.parse_file('../datasets/OpenFF-Industry-Benchmark-Season-1-v1.1-filtered-charge-coverage.json')

In [4]:
industry_benchmark_filter = industry_benchmark.filter(SMARTSFilter(smarts_to_include=['[r7:1]']))

In [5]:
industry_benchmark_rm = industry_benchmark_filter.to_records()

In [126]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolTransforms
from openff.toolkit import Molecule, ForceField, Topology
from openff.units import Quantity,unit

def get_angles(qm_rm,angle_data_dict,ff ,    angles_gt_cut,    cutoff=0.4):
    qm_mol = qm_rm[1]
    
    qm_mol_rd = qm_mol.to_rdkit()
    qm_conf = qm_mol_rd.GetConformer()
    smiles = qm_mol.to_smiles(mapped=True)
    
    topo = Topology.from_molecules([qm_mol])
    molecule_force_list = ff.label_molecules(topo) # dictionary of forces for the molecule, keys are type of force ('Bonds', 'Angles', etc)
    # these have the form {(atom1 idx, atom2 idx,...): FF parameter} for all bonds/angles/torsions in the molecule

    angle_dict = dict(molecule_force_list[0]['Angles'])

    for idx in angle_dict.keys():
        b = angle_dict[idx]

        # This part could potentially be eliminated by reading in QM geom data
        qm_ang = Chem.rdMolTransforms.GetAngleDeg(qm_conf,idx[0],idx[1],idx[2])

        if np.abs((qm_ang - b.angle.magnitude)/b.angle.magnitude) > cutoff: 
            if qm_rm not in angles_gt_cut:
                angles_gt_cut.append(qm_rm)

            if b.smirks in angle_data_dict:
                angle_data_dict[b.smirks]['molecules'].append(smiles)
                angle_data_dict[b.smirks]['envs'].append(idx)
                angle_data_dict[b.smirks]['qm_values'].append(qm_ang)
                # angle_data_dict[b.smirks]['mm_values'].append(mm_ang)
            else:
                angle_data_dict[b.smirks] = {'ident':b.id,
                                            'sage_value': b.angle.magnitude,
                                            'qm_values': [qm_ang],
                                            # 'mm_values': [mm_ang],
                                            'molecules': [smiles],
                                            'envs': [idx]}
                

In [7]:
sage = ForceField('openff_unconstrained-2.1.0.offxml')

# Closer look at the benchmark set

In [127]:
angle_data_dict_40 = {}
angles_gt_40=[]
for rm in industry_benchmark_rm:
    get_angles(rm,angle_data_dict_40,sage,angles_gt_40,cutoff=0.4)

angle_data_dict_25 = {}
angles_gt_25=[]
for rm in industry_benchmark_rm:
    get_angles(rm,angle_data_dict_25,sage,angles_gt_25,cutoff=0.25)

In [124]:
len(angles_gt_40)

42

In [80]:
angles_gt_40[6][1]

NGLWidget()

Manually examined the results

17, 18, 19, 20 is Sx4, covered elsewhere and the r7 rings are normal.

33-41 is epoxy--shouldn't be filtered out.

0-17, 21-33 are the deformed r7

In [79]:
len(angles_gt_25)

46

In [133]:
angles_gt_25[2][1]

NGLWidget()

In [88]:
angles_gt_25[2][1].to_smiles(mapped=True)

'[H:76][c:37]1[c:36]([c:35]([c:8]2[c:7]([c:38]1[H:77])[C:6]([C@:5]34[C@@:24]5([C@:11]3([C:12]([c:13]6[c:23]5[c:22]([c:16]([c:15]([c:14]6[H:52])[H:53])[C:17]([H:54])([H:55])[C:18]([H:56])([H:57])[C:19]7([C:20]([C:21]7([H:61])[H:62])([H:59])[H:60])[H:58])[H:63])([H:50])[H:51])[C:10]([C:9]2([H:46])[H:47])([H:48])[H:49])[N:25]=[C:26]([N:2]([C:3]([O:4]4)([H:42])[H:43])[C:1]([H:39])([H:40])[H:41])[N:27]([H:64])[C:28](=[O:29])[O:30][C:31]([C:32]([H:65])([H:66])[H:67])([C:33]([H:68])([H:69])[H:70])[C:34]([H:71])([H:72])[H:73])([H:44])[H:45])[H:74])[H:75]'

In [134]:
angles_gt_25[2][1].chemical_environment_matches('[r3:1]~[r3:2]~[#7;r7:3]')

[(4, 23, 24), (10, 23, 24)]

In [82]:
angle_data_dict_25

{'[*:1]~[#6X4:2]-[*:3]': {'ident': 'a1',
  'sage_value': 110.0631999136,
  'qm_values': [159.5802680971367,
   166.13137330974072,
   137.82939926218913,
   138.70407498612028,
   138.15104976808072,
   138.36458326559037,
   157.96026719162492,
   166.5307335033145,
   156.82804361902492,
   164.74264241750768,
   163.57631028717688,
   158.07007815227516,
   158.91559289834214,
   169.54698413154338,
   165.52542026517236,
   166.93070021035507,
   157.58990014069346,
   165.88018680637725,
   165.70997384730958,
   158.87667442904055,
   164.99219943393476,
   157.96704333575173,
   166.5738843446248,
   160.48489890283253,
   158.5743534943834,
   160.83953883174613,
   156.61085929214738,
   164.8536813633246,
   164.1905387796593,
   163.85509577920038,
   160.48819394169342,
   166.48331037166741,
   159.01473511481208],
  'molecules': ['[H:33][c:9]1[c:8]([c:7]([c:12]2[c:11]([c:10]1[H:34])[C:17]([N:16]([C:15]([C:14]([O:13]2)([H:35])[H:36])([H:37])[H:38])[H:39])([H:40])[H:41])[C:

25% cutoff--compared to 40% cutoff, 2 thru 5 are the additions. They seem fine, the geometric distortion is coming from the fused r3-r7 ring

In [59]:
r7_outliers = angles_gt_40[0:17] + angles_gt_40[21:33]

In [60]:
r7_outliers[0]

(OptimizationRecord(id=36959242, record_type='optimization', is_service=False, properties={}, extras={}, status=<RecordStatusEnum.complete: 'complete'>, manager_name='OpenFF_Mobley_HPC3-hpc3-l18-05-55f91995-0dee-4d70-b349-f06091368d87', created_on=datetime.datetime(2021, 4, 20, 14, 2, 58, 683991, tzinfo=datetime.timezone.utc), modified_on=datetime.datetime(2021, 7, 4, 4, 57, 0, 915712, tzinfo=datetime.timezone.utc), owner_user=None, owner_group=None, compute_history_=None, task_=None, service_=None, comments_=None, native_files_=None, specification=OptimizationSpecification(program='geometric', qc_specification=QCSpecification(program='psi4', driver=<SinglepointDriver.deferred: 'deferred'>, method='b3lyp-d3bj', basis='dzvp', keywords={'maxiter': 200, 'scf_properties': ['dipole', 'quadrupole', 'wiberg_lowdin_indices', 'mayer_indices']}, protocols=AtomicResultProtocols(wavefunction=<WavefunctionProtocolEnum.none: 'none'>, stdout=True, error_correction=ErrorCorrectionProtocol(default_poli

In [61]:
r7_ids = [rm[0].id for rm in r7_outliers]

In [62]:
np.savetxt('problem_ids/all_r7_outliers.txt',r7_ids,fmt='%s')